In [1]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import integrLie as lie

In [2]:
def spherical2cart(s):
    return np.array([np.cos(s[0])*np.cos(s[1]), np.cos(s[0])*np.sin(s[1]),np.sin(s[0])])

def cart2spherical(y):
    try:
        np.linalg.norm(y) < 1 + 1e-6 or np.linalg.norm(y) > 1 - 1e-6
    except:
        raise Exception('Not on the sphere')
    return np.array([np.arcsin(y[2]), np.arctan2(y[1],y[0])])

def rDistance(y,z):
    # return 2*np.arcsin(np.linalg.norm(y-z,2)/2)
    y = y.reshape(1,3)
    z = z.reshape(3,1)
    return np.arccos(y@z)

def ProjS2(p,q,t):
    return ((1-t)*p+t*q)/np.linalg.norm((1-t)*p+t*q)

In [3]:
def negSkwMatrix(x,D):
    x = x.reshape(3,1)
    f = ((np.eye(3)-x*np.transpose(x))@D)@x
    rslt = f*np.transpose(x)-x*np.transpose(f)
    return rslt

def vecField(x,D,C):
    rslt = (-negSkwMatrix(x,D)+C*lie.skw(x))@x
    return rslt

def energy(x,D):
    return 0.5*np.transpose(x)@D@x

In [4]:
# initialization
I1 = 0.5
I3 = 1.0
if I1 > I3:
    tmp = np.copy(I3)
    I3  = np.copy(I1)
    I1  = np.copy(tmp)
I2      = np.copy(I1)
diag    = np.array([1/I1, 1/I2, 1/I3]).reshape(3,)
Inertia = np.diag(diag)

C       = [0, 2, 5, 10, 20]
C_size  = len(C)

dt      = 0.1
my_time = np.arange(0, 5, dt)
N       = my_time.size

phi0   = - np.pi/2 + np.random.rand()
theta0 = 2*np.pi*np.random.rand()

q = np.zeros((C_size,3,N))
q[0,:,0] = spherical2cart([phi0,theta0])
for j in range(C_size-1):
    q[j+1,:,0] = np.copy(q[0,:,0])

In [5]:
now = datetime.datetime.now().strftime('%Y%m%dT%H%M%S')

for cc, the_c in enumerate(C):
    for cnt in range(N-1):
        q[cc,:,cnt+1] = lie.implieeul(lambda x: lie.skwinv(-negSkwMatrix(x, Inertia)+the_c*lie.skw(x)), q[cc,:,cnt], q[cc,:,cnt], dt)
        
np.save('../out/'+now+'_isotropy_sols',q)